In [1]:
import panel as pn
from panel.layout.gridstack import GridStack

import plotly.express as px
import pydeck as pdk

pn.extension('deckgl', 'plotly', sizing_mode='stretch_width')

# Widget utilities
number_widget = pn.indicators.Number(
    title_size='1em',
    font_size='2em',
    default_color='green',
    sizing_mode='stretch_width')

In [2]:
import pandas as pd
import json
from pathlib import Path
import datetime
import src.std_utils as std_utils

kDataPath = Path('data/agg')

def load_data():

    # Loads plants
    plants = std_utils.from_csv(kDataPath / 'plants.csv')
    plants_name = plants[['plant_code', 'plant_name']]

    # Loads data, merges with plants information (to get plants name)
    realtime = pd.merge(std_utils.from_csvs(
        kDataPath, '**/realtime.csv'), plants_name, on=['plant_code'])
    hourly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/hourly.csv'), plants_name, on=['plant_code'])
    daily = pd.merge(std_utils.from_csvs(
        kDataPath, '**/daily.csv'), plants_name, on=['plant_code'])
    monthly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/monthly.csv'), plants_name, on=['plant_code'])
    yearly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/yearly.csv'), plants_name, on=['plant_code'])
    alarms = pd.merge(std_utils.from_csvs(
        kDataPath, '**/alarms.csv'), plants_name, on=['plant_code'])

    return {'plants': plants,
            'realtime': realtime,
            'hourly': hourly,
            'daily': daily,
            'monthly': monthly,
            'yearly': yearly,
            'alarms': alarms}

data = load_data()

In [3]:
# Data utilities

def latest_date(data):
    return data['realtime']['collect_time'].max().to_pydatetime()


def latest_data(data):
    return data[data.groupby(['plant_code'])['collect_time'].transform(
        max) == data['collect_time']]


def data_in_range(data, begin, end):
    return data.loc[(data['collect_time'] >= begin)
                    & (data['collect_time'] <= end)]

In [4]:
def alarm_trends(data):
    severities = data['alarms']['alarm_severity'].value_counts()

    widgets = []
    for severity in ['Critical', 'Major', 'Minor', 'Warning']:
        widgets.append(number_widget.clone(name=severity, value=severities.get(severity, 0), colors=[(0, 'green'), (1e3, 'red')]))
        
    return widgets

pn.Row(*alarm_trends(data))

BokehModel(combine_events=True, render_bundle={'docs_json': {'f8f133f1-a3f1-4e50-941c-f63f00097b40': {'defs': …

In [42]:

def power_trends(data):
    latest = latest_date(data)

    def power_per_plant(data, end, duration):
        in_range = data_in_range(data, end - duration, end)
        dl = in_range.groupby('plant_name')['inverter_power'].sum().to_frame(
            name='total').reset_index()
        return dl

    widgets = []
    widgets.append(number_widget.clone(name='Capacity', value=data['plants']['capacity'].sum(), format='{value} kWp'))

    ds =power_per_plant(data['hourly'], latest, datetime.timedelta(1))['total'].sum()
    widgets.append(number_widget.clone(name='Yesterday', value=round(ds, 1), format='{value} kWh'))

    ms = power_per_plant(data['hourly'], latest, datetime.timedelta(30))['total'].sum()
    widgets.append(number_widget.clone(name='Monthly', value=round(ms/1000, 1), format='{value} MWh'))

    ys = power_per_plant(data['hourly'], latest, datetime.timedelta(365))['total'].sum()
    widgets.append(number_widget.clone(name='Yearly', value=round(ys/1000,1), format='{value} MWh'))

    lastest_realtime = latest_data(data['realtime'])
    widgets.append(number_widget.clone(name = 'Total', value = round(lastest_realtime['total_power'].sum()/1000,1), format = '{value} MWh'))
    
    return widgets

pn.Row(*power_trends(data))


BokehModel(combine_events=True, render_bundle={'docs_json': {'1f94159e-aea9-4fb5-939c-d0eadbe8a876': {'defs': …

In [43]:

def plot_qos(data):

    def qos_data(data):
        realtime = data['realtime']
        by_plant = realtime.groupby(['plant_name'])
        normalized = by_plant['health_state'].value_counts(normalize=True)
        count = by_plant['health_state'].value_counts()

        df = pd.DataFrame({'Quality of Service': normalized,
                        'Days': count}).reset_index()
        return df

    qos = qos_data(data)
    fig = px.bar(qos, x='plant_name', y='Quality of Service', color='health_state', hover_data=["Quality of Service", "Days"], orientation='v',
                 color_discrete_map=std_utils.health_state_colormap(), labels=std_utils.descriptions())
    fig.update_layout(xaxis_title=None)
    # ,config={'displayModeBar': False})

    return pn.pane.Plotly(fig)

plot_qos(data)


BokehModel(combine_events=True, render_bundle={'docs_json': {'fee62bed-9f01-4168-88a3-fa2b5c47c83b': {'defs': …

In [44]:
def realtime_plot(data):
    realtime = data['realtime'][['collect_time', 'total_power', 'day_power', 'month_power', 'plant_name']]
    fig = px.area(realtime, x='collect_time', y='total_power',
                    color='plant_name',
                    hover_data=['total_power', 'day_power', 'month_power'],
                    color_discrete_sequence=px.colors.qualitative.D3,
                    labels=std_utils.descriptions())
    fig.update_traces(mode="markers+lines")
    fig.update_layout(hovermode="x")
    fig.update_layout(xaxis_title=None)  # , config={'displayModeBar': False})

    return pn.pane.Plotly(fig)

realtime_plot(data)

BokehModel(combine_events=True, render_bundle={'docs_json': {'25fb7593-4787-43ff-ab91-9ca7f7416a32': {'defs': …

In [45]:

def plot_power_ctrl(data, step, past):
    # h = data['collect_time'].max().to_pydatetime()
    h = datetime.datetime.now()
    l = data['collect_time'].min()
    # l = min(data['collect_time'].min().to_pydatetime(), h - step)
    # l = h-step

    return {'min': l,
            'max': h,
            'rmin': max(l, h - past) if past else l,
            'rmax': h,
            'step': step}

def plot_power_data(data, begin, end):
    # in_range = data.loc[(data['collect_time'] >= pd.to_datetime(range[0], utc=True).date())
    #                     & (data['collect_time'] <= pd.to_datetime(range[1], utc=True).date())]
    #collect_date = data['collect_time'].dt.date
    in_range = data_in_range(data, begin, end)
    return in_range[['collect_time', 'plant_name', 'inverter_power']]

def _plot_power(data, step, past, slider_format, tickformat, dtick):
    ctrl = plot_power_ctrl(data, step, past)

    #range = date_range_picker(
    #    'Select date range', min_date=ctrl['min'], max_date=ctrl['max'], default_start=ctrl['rmin'], default_end=ctrl['rmax'], label_visibility='collapsed')

    # range = st.slider('Select date range', label_visibility='collapsed',
    #                   min_value=ctrl['min'], max_value=ctrl['max'],
    #                   value=(ctrl['rmin'], ctrl['rmax']),
    #                   step=ctrl['step'], format=slider_format)

    data_range = plot_power_data(data, ctrl['min'], ctrl['max'])

    fig = px.bar(data_range, x='collect_time', y='inverter_power', color="plant_name", #template='plotly_dark',
                 barmode='stack', labels=std_utils.descriptions())
    fig.update_layout(xaxis=dict(tickformat=tickformat, dtick=dtick))

    return pn.pane.Plotly(fig, sizing_mode='stretch_width')


def plot_power(data):
    widgets = []
    widgets.append(_plot_power(data['hourly'], datetime.timedelta(hours=1), datetime.timedelta(days=5), 'DD/MM/YYYY hh', '%Hh\n%d %b', ''))
    widgets.append(_plot_power(data['daily'], datetime.timedelta(days=1), datetime.timedelta(weeks=2*4), 'DD/MM/YYYY', '%d\n%b', '24*60*60*1000'))
    widgets.append(_plot_power(data['monthly'], datetime.timedelta(weeks=4), datetime.timedelta(weeks=2*52), 'MM/YYYY', '%b\n%Y', 'M1'))
    widgets.append(_plot_power(data['yearly'], datetime.timedelta(weeks=52), None, 'YYYY', '%Y', 'M12'))
    return widgets

pn.Row(*plot_power(data))


BokehModel(combine_events=True, render_bundle={'docs_json': {'d1d92525-d05e-4337-b959-84a2f6d33279': {'defs': …

In [46]:

def plant_map(data):

    def plant_map_data(data):
        realtime = data['realtime']
        # TODO move !!!
        realtime['month_power_m'] = realtime['month_power'].apply(lambda d: f'{round(d /1000, 1)}')
        realtime['total_power_m'] = realtime['total_power'].apply(lambda d: f'{round(d /1000, 1)}')
        
        lastest = realtime[realtime.groupby(['plant_code'])['collect_time'].transform(max) == realtime['collect_time']]

        return pd.merge(lastest, data['plants'], on=['plant_code', 'plant_name'])

    map_data = plant_map_data(data)

    boundaries = {}
    with open('resources/geo/territoire.geojson') as f:
        boundaries = json.load(f)

    geojson = pdk.Layer(
        'GeoJsonLayer',
        boundaries,
        opacity=0.8,
        stroked=True,
        filled=False,
        extruded=False,
        get_line_color=[0, 0, 255],
        line_width_min_pixels=2,
    )

    pv = pdk.Layer(
        'ColumnLayer',
        data=map_data,
        get_position='[longitude,latitude]',
        radius=80,
        auto_highlight=True,
        elevation_scale=50,
        pickable=True,
        opacity=0.5,
        get_fill_color='(health_state == "Healthy") ? [0, 255, 0, 255] : ((health_state == "Faulty") ? [255, 0, 0, 255]: [128, 128, 128, 255])',
        get_elevation='capacity',
        extruded=True)

    tooltip = {
        "html": "<b>{plant_name}</b><br>{plant_addr}<br>Capacity: {capacity} kWc<br>Status: {health_state}<br>Last day: {day_power} kWh<br>Last month: {month_power_m} MWh<br> Total: {total_power_m} MWh",
        "style": {
            "background": "grey",
            "color": "white",
            "z-index": "10000"}
    }

    view_state = pdk.ViewState(
        latitude=45.71,
        longitude=5.87,
        zoom=11,
        min_zoom=9,
        max_zoom=15,
        pitch=50,
        bearing=0
    )

    deck = pdk.Deck(map_style='road', layers=[geojson, pv], initial_view_state=view_state, tooltip=tooltip)
    return pn.pane.DeckGL(deck, sizing_mode='stretch_both')

plant_map(data)

BokehModel(combine_events=True, render_bundle={'docs_json': {'d4494273-b3ba-4a18-9a07-35e7072e8f4d': {'defs': …

app = pn.template.react.ReactTemplate(
    title='SuperCV', logo='resources/logo.png', theme=pn.template.react.ReactDarkTheme)

app.main[:,     0:1] = pn.Column(*alarm_trends(data))
app.main[0:1,   1:12] = pn.Row(*power_trends(data))
app.main[1:4,   1:12] = pn.Tabs(('Lifetime', realtime_plot(data)), *plot_power(data))
app.main[4:6,   7:12] = plant_map(data)
app.main[4:6,   1:6] = plot_qos(data)

app.show();


In [47]:
#app = pn.template.MaterialTemplate(title='SuperCV', logo='resources/logo.png')

stack = pn.GridSpec(sizing_mode='stretch_both')
#app.main.append(stack)
app = stack

stack[:,     0:1] = pn.WidgetBox(pn.Column(*alarm_trends(data)))
stack[0:1,   1:12] = pn.WidgetBox(pn.Row(*power_trends(data)))
stack[1:4,   1:12] = pn.WidgetBox(pn.Tabs(('Lifetime', realtime_plot(data)), *plot_power(data)))
stack[4:6,   7:12] = pn.WidgetBox(plant_map(data))
stack[4:6,   1:6] = pn.WidgetBox(plot_qos(data))

app.show()


Launching server at http://localhost:51362
